In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Beaconcha.in explorer api-key found in your account settings
eth_api_key = 'NmVSM2x3S3BUNFNsSnhvOFgxdFN1'

#validator indexes of validators you wish to follow
validator_ids = ['310494', '356165', '356169','410931']

In [3]:
#test Beaconcha.in explorer REST API
request_string = 'https://beaconcha.in/api/v1/validator/' + validator_ids[0] + '?apikey=' + eth_api_key
r = requests.get(request_string)

In [4]:
#inspect results
val_data = r.json()

In [5]:
#inspect json file
print(val_data)

{'status': 'OK', 'data': {'activationeligibilityepoch': 104354, 'activationepoch': 104730, 'balance': 32841123550, 'effectivebalance': 32000000000, 'exitepoch': 9223372036854775807, 'lastattestationslot': 4789236, 'name': None, 'pubkey': '0xb12021412d40a255518701b59c75ef4fb87ebbf5fddae629253370b5c68561c6cebc5bdd364689ead2742ecfc0d47ea4', 'slashed': False, 'status': 'active_online', 'validatorindex': 310494, 'withdrawableepoch': 9223372036854775807, 'withdrawalcredentials': '0x00127e0d231f1fdcd1de2ed59ea0b87c5c18db6ddbee25aaa24137d6c85761c1'}}


In [6]:
print(val_data['data']['balance'])

32841123550


#### We now see how to extract the data we want from the JSON file returned from the API call 
#### Next, we will loop through all validators and calculate earnings to date

In [7]:
#calculate the earnings to date for each validator
validator_earnings = {}
for val_id in validator_ids:
    #create request string
    request_string = 'https://beaconcha.in/api/v1/validator/' + val_id + '?apikey=' + eth_api_key
    
    #make API call
    r = requests.get(request_string)
    
    #extract data from request
    val_data = r.json()
    
    #calculate the lifetime earnings in ETH
    val_earn = val_data['data']['balance']/1000000000 - 32
    
    #append earnings
    validator_earnings[val_id] = val_earn

In [8]:
#check to make sure validator earnings look accurate
print(validator_earnings)

{'310494': 0.8411235499999989, '356165': 0.5168724299999994, '356169': 0.6544393089999971, '410931': 0.23579301899999905}


#### Now that we have earning for all of the validators, we will calculate the fees taking by stake pool operator (10%) and then calcuate the ETH balance for all validator stakers

In [9]:
#list of stake pool contributers
stakers_list = ['Mr. Lites','Dr. Hutchinson','Matt','Nader','Kairn']

#add amount of stake for each contributer per validator 
ownership = {validator_ids[0]:{'Mr. Lites':32}, 
             validator_ids[1]:{'Mr. Lites':32}, 
             validator_ids[2]:{'Dr. Hutchinson':32},
             validator_ids[3]:{'Mr. Lites':10,'Matt':14,'Nader':4,'Kairn':4}}

In [10]:
#calculate stake operater earnings 
pool_operator_earnings = sum(validator_earnings.values()) * 0.1
print('Pool operater earnings: ' + str(pool_operator_earnings))

#calculate earnings to be split amongst stakers
pool_participant_earnings = sum(validator_earnings.values()) - pool_operator_earnings
print('Pool participant earnings: ' + str(pool_participant_earnings))


Pool operater earnings: 0.22482283079999946
Pool participant earnings: 2.023405477199995


In [11]:
#calulate the balance after fees of each staker

#first we will loop through our stakers
eth_balance = {}
for staker in stakers_list:
    
    #check to see if staker is an owner in each validator. If so, extract their initial eth stake
    total_eth = 0
    for validator in ownership:
        if staker in ownership[validator]:
            initial_stake = ownership[validator][staker]
            stake_percentage = initial_stake/32
            earnings_share = stake_percentage * validator_earnings[validator] * 0.9
            val_balance = earnings_share + initial_stake
            total_eth = total_eth + val_balance
    eth_balance[staker] = total_eth

In [12]:
#sanity check the results
check1 = sum(eth_balance.values()) + pool_operator_earnings
print(check1)

130.248228308


In [13]:
check2 = 4 * 32 + sum(validator_earnings.values())
print(check2)

130.248228308


In [14]:
for eth_staker in eth_balance:
    print(eth_staker + ' : ' + str(eth_balance[eth_staker]))

Mr. Lites : 75.28851316859374
Dr. Hutchinson : 32.588995378099995
Matt : 14.09284350123125
Nader : 4.0265267146375
Kairn : 4.0265267146375


### Next Steps
#### Factor in the price of ETH and show dollar amounts
#### Make this run on terminal 
#### Use flask to create dashboard 
#### Dockerize

In [15]:
#make call to Binance API for ETH_price
price_request_string = "https://api.binance.com/api/v3/ticker/price?symbol=ETHUSDT"

r2 = requests.get(price_request_string)

data = r2.json()

eth_price = float(data['price'])
print(eth_price)

1353.13


In [16]:
#calculate usd balance
usd_balance = {}
for staker in eth_balance:
    usd = eth_balance[staker] * eth_price
    usd_balance[staker] = usd
print(usd_balance)

{'Mr. Lites': 101875.14582381926, 'Dr. Hutchinson': 44097.14731596845, 'Matt': 19069.449326821043, 'Nader': 5448.414093377441, 'Kairn': 5448.414093377441}


In [17]:
eth_price

1353.13